<center><h1><b><span style="color:blue">Files, data, event processing</span></b></h1></center>

#### **Quick intro to the following packages**
- `uproot`.
- `awkward-array`.

&nbsp;
<center>
    <img src="images/logo_uproot.png" alt="uproot package logo" style="width: 200px;"/>
    <h2><b><span style="color:green">ROOT I/O in pure Python and NumPy</span></b></h2>
</center>

**What is ``uproot``?**

`uproot` provides very fast, efficient, and convenient access to ROOT trees.

- Pure Python + NumPy implementation of ROOT I/O (until recently, only “input”)
- An array-centric view of ROOT TTree data:
  - branches of simple types are simple arrays
  - branches of complex types are “jagged arrays”
-  High performance for large baskets, despite Python’s slowness (because all per-entry operations are performed in Numpy)
- Greatest benefits: simplicity, minimal installation, set-up, and affinity with machine learning interfaces

<center><img src="images/uproot_performance.png" alt="uproot package performance" width = "75%"/></center>

### **1. Getting data from a ROOT TTree**

In [ ]:
import uproot

In [ ]:
f = uproot.open('data/toy-Z2mumu.root')

f

In [ ]:
f.keys()

In [ ]:
t = f[b'events']
t.keys()

In [ ]:
t['M']

In [ ]:
t['M'].array()

In [ ]:
t.arrays()

In [ ]:
t.arrays(['Run', 'Event'])

All branches can be looked at with `t.arrays()` ...

You can now start performing calculations, e.g. (not necessary the best way, think NumPy type of operations ;-)).

In [ ]:
import numpy

for px,py,pz in t.iterate(["px1","py1","pz1"], outputtype=tuple):
    pt = numpy.sqrt(px**2 + py**2)
    eta = numpy.arctanh(pz / numpy.sqrt(px**2 + py**2 + pz**2))
    phi = numpy.arctan2(py, px)
    print(pt)
    print(eta)
    print(phi)

In [ ]:
import pandas

df = t.pandas.df()
df

In [ ]:
t.pandas.df(['Run', 'Event', 'pt1', 'pt2']).head()

### **2. Data with jagged structure**

In [ ]:
f = uproot.open('data/uproot-tutorial-file.root')

f.keys()

In [ ]:
branches = f[b'Events'].arrays(namedecode='utf-8')
                               
branches

In [ ]:
branches['nMuon']

In [ ]:
branches['Muon_pt']

In [ ]:
print(' \n'.join([str(elm) for elm in branches['Muon_pt'][:10]]))

We will get back to jagged arrays in a sec. Let's first show that `uproot` also has (limited) writing functionality.

### **3. Interoperability - writing files with uproot**

`uproot` version 3 started support for writing files!

In [ ]:
f = uproot.recreate("tmp.root")

In [ ]:
f["name"] = numpy.histogram(numpy.random.normal(0, 1, 100000), bins=20)

In [ ]:
f.closed

In [ ]:
f.keys()

In [ ]:
ar = f["name"]

ar.show()

But ... can the file actually be read back in ROOT ...?

In [ ]:
import ROOT

f = ROOT.TFile("tmp.root")
h = f.Get("name")

c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)
h.Draw('hist')
c.Draw()

Yes it can!

&nbsp;<br><center><img src="images/logo_awkward_array.png" alt="awkward-array package logo" style="width: 200px;"/></center>

<center><h2><b><span style="color:green">Manipulate arrays of complex data structures as easily as Numpy</span></b></h2></center>

In [ ]:
import awkward

In [ ]:
branches

In [ ]:
table = awkward.Table(branches)

table

In [ ]:
table.nMuon[0]

In [ ]:
branches['nMuon'][0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(table.nMuon, bins=10, range=(0, 10))
plt.xlabel('Number of muons in event')
plt.ylabel('Number of events');

In [ ]:
branches['nMuon'].sum()

In [ ]:
len(branches['Muon_pt'].flatten())

In [ ]:
plt.hist(table.Muon_pt.flatten(), bins=100, range=(0, 100))
plt.xlabel('Muon pT')
plt.ylabel('Number of events')
plt.yscale('log');

In [ ]:
plt.hist(table.Muon_eta.flatten(), bins=100, range=(-2.5, 2.5))
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of events');

In [ ]:
print(len(t))
print(len(table))
print(len(branches['nMuon']))
print(len(branches['Muon_pt'])) # or any of the other branches...

In [ ]:
%%timeit
len(branches['Muon_pt'].flatten())

In [ ]:
%%timeit
branches['nMuon'].sum()

In [ ]:
branches['nMuon'] == 1

In [ ]:
single_muon_mask = branches['nMuon'] == 1
single_muon_mask.sum()

In [ ]:
branches['Muon_pt'][single_muon_mask]

In [ ]:
plt.hist(branches['Muon_pt'][single_muon_mask].flatten(), bins=100, range=(0, 100))
plt.xlabel('Muon $p_{\mathrm{T}}$ [MeV]')
plt.ylabel('Number of single muons / 1 MeV')
plt.yscale('log')
plt.show()

In [ ]:
eta_mask = abs(branches['Muon_eta']) < 2
eta_mask

In [ ]:
eta_mask.flatten().sum()

In [ ]:
eta_mask.sum()

In [ ]:
eta_mask.sum().sum()

In [ ]:
plt.hist(branches['Muon_eta'].flatten(), bins=50, range=(-2.5, 2.5))
plt.title('No selection')
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.show()

plt.hist(branches['Muon_eta'][eta_mask].flatten(), bins=50, range=(-2.5, 2.5))
plt.title('With $|\eta| < 2$ selection')
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.show()

In [ ]:
len(single_muon_mask & eta_mask)

In [ ]:
plt.hist([branches['Muon_pt'][single_muon_mask & eta_mask].flatten(),
          branches['Muon_pt'][single_muon_mask & ~eta_mask].flatten()],
         label=['$|\eta| < 2$', '$|\eta| \geq 2$'],
         density=True,
         bins=25, range=(0, 50))
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.ylabel('Number of single muons / 2 GeV')
plt.legend()
plt.show()

In [ ]:
two_muons_mask = branches['nMuon'] == 2
two_muons_table = table[two_muons_mask]

In [ ]:
import uproot_methods

two_muons_p4 = uproot_methods.TLorentzVectorArray.from_ptetaphim(two_muons_table['Muon_pt'],
                                                                 two_muons_table['Muon_eta'],
                                                                 two_muons_table['Muon_phi'],
                                                                 two_muons_table['Muon_mass'])
two_muons_p4

In [ ]:
len(two_muons_p4)

In [ ]:
first_muon_p4 = two_muons_p4[:, 0]
second_muon_p4 = two_muons_p4[:, 1]

In [ ]:
first_muon_p4.delta_r(second_muon_p4)

In [ ]:
plt.hist(first_muon_p4.delta_r(second_muon_p4), bins=100)
plt.xlabel('$\Delta R$ between muons')
plt.ylabel('Number of two-muon events')
plt.show()

In [ ]:
sum_p4 = first_muon_p4 + second_muon_p4
opposite_sign_muons_mask = two_muons_table['Muon_charge'][:, 0] != two_muons_table['Muon_charge'][:, 1]
dimuon_p4 = sum_p4[opposite_sign_muons_mask]
dimuon_p4

In [ ]:
import numpy as np

figsize_l, figsize_h = plt.rcParams["figure.figsize"]
plt.figure(figsize=(figsize_l*2.5, figsize_h*3.))

(yvals, binedges, patches) = plt.hist(dimuon_p4.mass, bins=np.logspace(np.log10(0.1), np.log10(1000), 200))

plt.xlabel('Dimuon invariant mass [GeV]')
plt.ylabel('Number of dimuon events')
plt.xscale('log')
plt.yscale('log')

import particle.particle.literals as lpart
from hepunits import GeV
    
list_particles = [getattr(lpart,name) for name in ('eta', 'rho_770_0', 'omega_782','phi_1020','Jpsi_1S', 'Z_0')]

for p in list_particles:
    binnumber = np.searchsorted(binedges, p.mass/GeV)
    plt.text(p.mass/GeV, yvals[binnumber-1]*1.02, '${}$'.format(p.latex_name), horizontalalignment='center', fontsize=16)

plt.show()

&nbsp;
<div class="alert alert-warning">

<b>Important note:</b>

These 2 packages are currently under a major overhaul!
</div>

- `awkward-array` will soon-ish be replaced by an improved packaged nicknamed `awkward-1.0` for now. Developments are happening on [this GitHub repository](https://github.com/scikit-hep/awkward-1.0).
- `uproot` version 4 will be the upgrade version of `uproot` version 3 built atop `awkward-1.0`.